In [1]:
import cv2
import numpy as np
import torch
from collections import OrderedDict
import os
import glob
from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
import itertools
import math
import time
from IPython import display
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchvision.utils as vutils
from torch import optim
from IPython import display

import os
import cv2

src = "dog/Biggle" #pokeRGB_black
dst = "resizedData/Biggle" # resized


os.mkdir(dst)

for each in os.listdir(src):
    img = cv2.imread(os.path.join(src,each))
    img = cv2.resize(img,(64,64))
    cv2.imwrite(os.path.join(dst,each), img)   

In [2]:
dir_path = 'newresizedData/'
nz = 100 # 노이즈 벡터의 크기
nc = 3 # 채널의 수
ngf = 64 # generator 필터 조정
ndf = 64 # discriminator 필터 조정
epochNum = 2000 # 에폭 수
lr = 0.0002
beta1 = 0.5

imageSize = 64 # 만들어지는 이미지의 크기
batchSize = 1 # 미니배치의 크기
outf = "making/fake"
outf1 = "making/real"

In [3]:
class poketmon(Dataset):
    def __init__(self, path, transform=None):
        self.classes   = os.listdir(path)
        self.path      = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*.jpg") for x in self.path]
        self.transform = transform
        files = []
        print(self.transform)
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
        self.file_list = files
       
        
        files = None
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
        return im, classCategory

In [4]:
mean = [0.5, 0.5, 0.5]
std  = [0.5, 0.5, 0.5]
transform = transforms.Compose([
                                #transforms.Resize((160,160)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])

In [5]:
train_dataset = poketmon(dir_path,transform=transform)

Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)


In [6]:
train_loader = DataLoader(dataset = train_dataset,
                         shuffle = True,
                         batch_size = batchSize)
print(train_loader)

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:         # Conv weight init
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:  # BatchNorm weight init
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [8]:
class _netG(nn.Module):
    def __init__(self):
        super(_netG, self).__init__()
        self.main = nn.Sequential(

            # 입력값은 Z이며 Transposed Convolution을 거칩니다.
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            # (ngf * 8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            # (ngf * 4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            # (ngf * 2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            # ngf x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, input):
        output = self.main(input)
        return output

In [9]:
class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # (nc) x 64 x 64)
            nn.Conv2d(nc, ndf, 4,2,1,bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # ndf x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # (ndf * 2) x 16 x 16
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # (ndf * 4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)

In [10]:
netG = _netG().cuda()
netG.apply(weights_init)
print(netG)

netD = _netD().cuda()
netD.apply(weights_init)
print(netD)
criterion = nn.BCELoss()

input = torch.FloatTensor(batchSize, 3, imageSize, imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
fixed_noise = Variable(fixed_noise).cuda()

label = torch.FloatTensor(batchSize)
real_label = 1
fake_label = 0

_netG(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)
_netD(
  (main): Sequent

In [11]:
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
for epoch in range(epochNum):
    for i, data in enumerate(train_loader):
        # train with real
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)

        input.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(batch_size).fill_(real_label)

        inputv = Variable(input).cuda()
        labelv = Variable(label).cuda()

        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        # train with fake
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise).cuda()
        fake = netG(noisev)
        labelv = Variable(label.fill_(fake_label)).cuda()
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()

        errD = errD_real + errD_fake
        optimizerD.step()

        
        
        netG.zero_grad()
        labelv = Variable(label.fill_(real_label)).cuda()
        output = netD(fake)

        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()
        if ((i+1) % 100 == 0):
            print(i, "step")
            fake = netG(fixed_noise)
            
            vutils.save_image(fake.data,'%s/fake_samples_epoch_%s.png' % (outf, str(epoch)+" "+str(i+1)),normalize=True)
            
    vutils.save_image(real_cpu,'%s/real_samples.png' % outf1,normalize=True)
    
    fake = netG(fixed_noise)
    
    vutils.save_image(fake.data,'%s/fake_samples_epoch_%s.png' % (outf, epoch),normalize=True)
    
    result_dict = {"loss_D":errD,"errD_fake":errD_fake }
    if (epoch+1) % 100 ==0:
        print(epoch)

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG.pth' % (outf))
    torch.save(netD.state_dict(), '%s/netD.pth' % (outf1))

99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 step
99 step
199 st